<a href="https://colab.research.google.com/github/andrewwhite5/DS-Unit-2-Kaggle-Challenge/blob/master/SprintChallenge/DS_Unit_2_Sprint_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [0]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install packages in Colab
    !pip install category_encoders==2.0.0
    !pip install pandas-profiling==2.3.0
    !pip install plotly==4.1.1

In [0]:
import pandas as pd

# Read data
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)

# Check data shape
assert df.shape == (13958, 20)

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?

**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.

**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 

**7.** Get your model's **test accuracy.** (One time, at the end.)


**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**

### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.



## 1. Begin with baselines for classification. 

>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [3]:
majority_class = df['shot_made_flag'].value_counts(normalize=True)[0]
majority_class

0.5270812437311936

**Baseline accuracy would be 0.527**

## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [4]:
df.columns.unique()

Index(['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type', 'shot_type',
       'shot_zone_basic', 'shot_zone_area', 'shot_zone_range', 'shot_distance',
       'loc_x', 'loc_y', 'shot_made_flag', 'game_date', 'htm', 'vtm',
       'season_type', 'scoremargin_before_shot'],
      dtype='object')

In [5]:
type(df['game_date'].loc[0])

str

In [0]:
df['game_date'] = pd.to_datetime(df['game_date'])

In [7]:
print(df.shape)
df.head()

(13958, 20)


,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0


In [0]:
import datetime

start_date = datetime.datetime(2018,10,1)
end_date = datetime.datetime(2019,6,30)

test_mask = (df['game_date'] >= start_date) & (df['game_date'] <= end_date)
train_mask = (df['game_date'] < start_date) | (df['game_date'] > end_date)

In [9]:
# Train should be 12249
# Test should be 1709

test = df.loc[test_mask]
train = df.loc[train_mask]

print(train.shape, test.shape)

(12249, 20) (1709, 20)


## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

    

In [47]:
import numpy as np

train['homecourt_advantage'] = np.where(train['htm'] == 'GSW', 'True', 'False')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## **4. Decide how to validate** your model. 

>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [0]:
# Random .80/.20 train/val split
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.8, test_size=0.2)

In [51]:
print(train.shape, val.shape)

(9799, 21) (2450, 21)


## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [70]:
train.describe()

,game_id,game_event_id,period,minutes_remaining,seconds_remaining,shot_distance,loc_x,loc_y,shot_made_flag,scoremargin_before_shot
count,9.799000e+03,9799.000000,9799.000000,9799.000000,9799.000000,9799.000000,9799.000000,9799.000000,9799.000000,9799.000000
mean,2.418781e+07,261.370854,2.409940,4.754057,28.359833,17.394734,0.221247,130.038984,0.471375,1.468823
std,7.038141e+06,163.700208,1.125711,3.350080,17.597126,10.251258,123.288514,101.956249,0.499205,10.176771
min,2.090002e+07,2.000000,1.000000,0.000000,0.000000,0.000000,-250.000000,-36.000000,0.000000,-39.000000
25%,2.120055e+07,105.000000,1.000000,2.000000,13.000000,8.000000,-92.000000,23.000000,0.000000,-5.000000
50%,2.140081e+07,268.000000,2.000000,4.000000,29.000000,21.000000,1.000000,140.000000,0.000000,1.000000
75%,2.160086e+07,384.000000,3.000000,7.000000,43.000000,25.000000,94.000000,216.000000,1.000000,8.000000
max,4.170040e+07,712.000000,6.000000,11.000000,59.000000,83.000000,247.000000,811.000000,1.000000,43.000000


In [67]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
player_name,9799,1,Stephen Curry,9799,NaN,NaN
shot_type,9799,2,2PT Field Goal,5157,NaN,NaN
season_type,9799,2,Regular Season,8407,NaN,NaN
homecourt_advantage,9799,2,False,4913,NaN,NaN
shot_zone_range,9799,5,24+ ft.,4573,NaN,NaN
shot_zone_area,9799,6,Center(C),4263,NaN,NaN
shot_zone_basic,9799,7,Above the Break 3,3895,NaN,NaN
htm,9799,32,GSW,4886,NaN,NaN
vtm,9799,32,GSW,4913,NaN,NaN
action_type,9799,51,Jump Shot,4738,NaN,NaN


In [0]:
# Arrange data into X features matrix and y target vector 

target = 'shot_made_flag'
# cat_features = ['shot_type', 'season_type', 'homecourt_advantage', 
#                 'shot_zone_range', 'shot_zone_area', 'shot_zone_basic']
# num_features = ['period', 'minutes_remaining', 'seconds_remaining', 
#                 'shot_distance', 'loc_x', 'loc_y', 'scoremargin_before_shot']
# features = cat_features + num_features
features = ['shot_type', 'season_type', 'shot_zone_range', 'shot_zone_area', 
            'shot_zone_basic', 'period', 'minutes_remaining', 
            'seconds_remaining', 'shot_distance', 'loc_x', 'loc_y', 
            'scoremargin_before_shot']

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [84]:
# Create pipeline

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline (
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    DecisionTreeClassifier(random_state=42)
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['shot_type', 'season_type',
                                     'shot_zone_range', 'shot_zone_area',
                                     'shot_zone_basic'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', return_df=True,
                               use_cat_names=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=...
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(class_weight=None, criterion='gini',
                                        max_depth=None, max_features=None,
                                        max_leaf_nodes=None,
                                        

In [0]:
y_pred = pipeline.predict(X_test)

## 6.Get your model's validation accuracy

> (Multiple times if you try multiple iterations.)

In [86]:
print ('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.5346938775510204


## 7. Get your model's test accuracy

> (One time, at the end.)

In [91]:
print ('Test Accuracy', pipeline.score(X_test, y_test))

Test Accuracy 0.5184318314803978


## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [87]:
correct_predictions = 85 + 36
total_predictions = correct_predictions + 8 + 58
accuracy = correct_predictions / total_predictions
print('Accuracy:', accuracy)

Accuracy: 0.6470588235294118


### Calculate precision

In [89]:
# Precision for Negative
correct_predictions_negative = 85
total_predictions_negative = 85 + 8
precision_negative = correct_predictions_negative / total_predictions_negative

# Precision for Positive
correct_predictions_positive = 36
total_predictions_positive = 36 + 58
precision_positive = correct_predictions_positive / total_predictions_positive

# Precisions
print('Precision (Negative):', precision_negative)
print('Precision (Positive):', precision_positive)

Precision (Negative): 0.9139784946236559
Precision (Positive): 0.3829787234042553


### Calculate recall

In [90]:
# Actuals
actual_negative = 85 + 58
actual_positive = 8 + 36

# Recall
recall_negative = correct_predictions_negative / actual_negative
recall_positive = correct_predictions_positive / actual_positive
print('Recall (Negative):', recall_negative)
print('Recall (Positive):', recall_positive)

Recall (Negative): 0.5944055944055944
Recall (Positive): 0.8181818181818182
